In [0]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data', one_hot=True)

In [0]:
batch_size = 200
m_batch = mnist.train.num_examples // batch_size

def variable_summaries(var):
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean) # 平均值
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev) ## 标准差
    tf.summary.scalar('max', tf.reduce_max(var)) ## 最大值
    tf.summary.scalar('min', tf.reduce_min(var)) ## 最小值
    tf.summary.histogram('histogram', var) ## 直方图


with tf.name_scope('input'):
  x = tf.placeholder(tf.float32, shape=[None, 784], name='input_x')
  y = tf.placeholder(tf.float32, shape=[None, 10], name='input_y')

with tf.name_scope('layer'):
  with tf.name_scope('weights'):
    w = tf.Variable(tf.zeros(shape=[784, 10]), name='weight_1')
    variable_summaries(w)
  with tf.name_scope('bias'):
    b = tf.Variable(tf.zeros(shape=[10], name='bias_1'))
    variable_summaries(w)
  with tf.name_scope('matmul'):
    r = tf.matmul(x, w) + b
  with tf.name_scope('softmax'):
    prediction = tf.nn.softmax(r)

with tf.name_scope('loss'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
  labels=y, logits=prediction))
  tf.summary.scalar('loss', loss)

with tf.name_scope('train'):
  train = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
  with tf.name_scope('correct_prediction'):
    correct_prediction = tf.equal(tf.argmax(y, axis=1),
                tf.argmax(prediction, axis=1))

  with tf.name_scope('acc'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('loss', accuracy)

# 合并所有summary

merged = tf.summary.merge_all()

with tf.Session() as sess:
  sess.run(init)
  writer = tf.summary.FileWriter('logs/', sess.graph)
  for epoch in range(200):
    for batch in range(m_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      _, _summary, loss_val = sess.run([train, merged, loss], feed_dict={x:batch_xs, y:batch_ys})
    writer.add_summary(_summary, epoch)
    acc_val = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
    print('%d epoch, Test accuracy %f' % (epoch, acc_val))